This notebook maps SDWIS, USGS GU and USGS model predictions data sets

In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

'sdwis' includes data from 4 quarters. Dupicated for PWS ID, Facility ID and treatment process were dropped. Included facilities with Facility Activity Status = Active/Inactive

USGS GUs are based on places- Has assigned PWS IDs/Seller PWS IDs and WSA IDs

DPC has WSA 

Goal of this flow sheet is to map PWS ID from the SDWIS data set to the WSA ID using the USGS GU data set. Map the per capita consumption to the WSA ID using the DPC data set

In [8]:
# df_latest_2 has sdwis data for all 4 quarters and drops duplicates
PATH = r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024'
sdwis = pd.read_csv(PATH + '\df_latest_2.csv',low_memory=False)
# Reading the USGS v1_GU_wWS file and converting to a dataframe
usgs_gu_wwsa = gp.read_file(PATH + r'\V1_GU_wWS\v1_GU_wWS.shp')
usgs_gu_wwsa_df = pd.DataFrame(usgs_gu_wwsa.drop(columns='geometry'))

# Reading Machine learning model output for domestic demand per capita and filtering for the year 2020
dpc_2020 = pd.read_csv(PATH + r'\delivery_water_use_model\predictions\national_dpc_predictions.csv')
dpc_2020 = dpc_2020.loc[dpc_2020['year']==2020].copy()

In [25]:
# Drop rows with duplicate PWS ID, Facility ID and treatment process
sdwis_filtered = sdwis[['PWS ID','Facility Id','PWS Name', 'Population Served Count', 
                        'Activity Status', 'Facility Activity', 'Primary Source', 'PWS Type',
                        'Treatment Process','Treatment Objective']].drop_duplicates(subset = ['PWS ID','Facility Id','Treatment Process'])

# Remove rows without an treatment process listed
sdwis_filtered = sdwis_filtered.dropna(subset=['Treatment Process'])
# Remove PWS ID and Facility with only 1 treatment process
# sdwis_filtered = sdwis_filtered.groupby(['PWS ID','Facility Id']).filter(lambda x: len(x['Treatment Process'])>1).reset_index()

# Filter for where population served is 0
# sdwis_filtered = sdwis_filtered[sdwis_filtered['Population Served Count']>0]
# usgs_gu_wwsa_df  = usgs_gu_wwsa_df[usgs_gu_wwsa_df['GU_POP']>0]

# Update the USGS DPC data to be upper case if it is alphanumeric
updated_column_list = []
avg_dpc = []

for idx,row in dpc_2020.iterrows():
    try:
        a = int(row['wsa_agidf'])
        updated_column_list.append(str(f'{a:09d}'))
    except:
        updated_column_list.append(row['wsa_agidf'].upper())

    avg_dpc.append(row.iloc[5::].mean())

dpc_2020['wsa_agidf'] = updated_column_list
dpc_2020['avg_dpc'] = avg_dpc

In [26]:
# Get unique PWS IDs in SDWIS data set
sdwis_pws_ids = sdwis_filtered['PWS ID'].unique()

# Get unique PWS IDs/Seller PWS ID in USGS GU dataset 
usgs_pws_ids = usgs_gu_wwsa_df['PWS_ID'].unique()
usgs_seller_pws_ids = usgs_gu_wwsa_df['SELLER_PWS'].dropna().unique()

combined_pws_ids = np.unique(np.concatenate([usgs_seller_pws_ids,usgs_pws_ids]))
# Get unique WSA ID in USGS GU dataset
usgs_wsa_ids = usgs_gu_wwsa_df['WSA_AGIDF'].unique()
# Get unique WSA ID in USGS DPC dataset
dpc_wsa_ids = dpc_2020['wsa_agidf'].unique()

print('Unique PWS_ID in SDWIS filtered (df_latest_2):', len(sdwis_pws_ids))


print('\nUSGS v1_GU-wWS')
print('Total USGS v1_GU-wWS rows:',len(usgs_gu_wwsa_df))
print('Unique PWS_ID in USGS v1_GU-wWS:', len(usgs_pws_ids))
print('Unique Seller PWS_ID in USGS v1_GU-wWS:', len(usgs_seller_pws_ids))
print('Total unique PWS IDs (with/without seller) in v1_USGS GU-wWS:', len(combined_pws_ids))
print('Unique WSA_AGIDF in USGS v1_GU-wWS:', len(usgs_wsa_ids))
print('\nUSGS DPC data:', len(dpc_wsa_ids))

Unique PWS_ID in SDWIS filtered (df_latest_2): 53130

USGS v1_GU-wWS
Total USGS v1_GU-wWS rows: 27159
Unique PWS_ID in USGS v1_GU-wWS: 21429
Unique Seller PWS_ID in USGS v1_GU-wWS: 1706
Total unique PWS IDs (with/without seller) in v1_USGS GU-wWS: 22252
Unique WSA_AGIDF in USGS v1_GU-wWS: 18807

USGS DPC data: 18807


In [27]:
# Check if all the WSA IDs are the same between the USGS GU-wWS and USGS DPC dataset
res_usgs_wsa_ids = [i for i in usgs_wsa_ids if i not in dpc_wsa_ids]
res_dpc = [i for i in dpc_wsa_ids if i not in usgs_wsa_ids]

print('In USGS v1_GU_wWs dataset but not USGS DPC:', res_usgs_wsa_ids)
print('In USGS DPC dataset but not USGS v1_GU_wWs:', res_dpc)

In USGS v1_GU_wWs dataset but not USGS DPC: ['CO0118006', 'CA3610057', 'CO0203002']
In USGS DPC dataset but not USGS v1_GU_wWs: ['CA2010009', 'CA3310018', 'CA3310025']


In [28]:
# Trying to check if USGS v1_GU_wWs dataset PWS IDs can be mapped to SDWIS df_latest dataset

usgs_check = usgs_gu_wwsa_df[['GU_ID','PLACE_FIPS','PLACE_NAME','CNTY_NM', 'STATE_NAME','PWS_ID','SELLER_PWS','WSA_AGIDF']].copy()

# Check if the PWS ID or Seller PWS ID is in the SDWIS dataset
check_list = []
pws_check_list = []

for idx,row in usgs_check.iterrows():
    if row['SELLER_PWS'] in sdwis_pws_ids:
        check_list.append('Yes')
        pws_check_list.append(row['SELLER_PWS'])
    elif row['PWS_ID'] in sdwis_pws_ids:
        check_list.append('Yes')
        pws_check_list.append(row['PWS_ID'])
    elif row['WSA_AGIDF'] in sdwis_pws_ids:
        check_list.append('Yes')
        pws_check_list.append(row['WSA_AGIDF'])
    else:
        check_list.append('No')
        pws_check_list.append(row['WSA_AGIDF'])

usgs_check['pws_check'] = pws_check_list
usgs_check['In SDWIS'] = check_list


In [29]:
# Mapping usgs check pws_check to dpc 2020 population
usgs_check_dpc = usgs_check.merge(dpc_2020[['wsa_agidf','avg_dpc']],left_on='WSA_AGIDF',right_on='wsa_agidf')
# usgs_check_dpc.to_csv(r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024\usgs_check_dpc.csv')

Results Summary

In [30]:
print('USGS v1_GU_wWs PWS IDs total:', len(combined_pws_ids))

print('\nNumber of rows in USGS v1_GU_wWs that were mapped to SDWIS:', len(usgs_check[usgs_check['In SDWIS']=='Yes']))
print('Number of rows in USGS v1_GU_wWs that were NOT mapped to SDWIS:', len(usgs_check[usgs_check['In SDWIS']=='No']))

usgs_check_yes = usgs_check[usgs_check['In SDWIS']=='Yes'].copy()
usgs_check_no = usgs_check[usgs_check['In SDWIS']=='No'].copy()
print('Number of unique PWS IDs/WSA IDs of the ones that were mapped to SDWIS:',len(usgs_check['pws_check'].unique()))
# print('Number of unique PWS IDs of the ones that were mapped to the SDWIS:',len(usgs_check_yes['pws_check'].unique()))
print('Number of unique PWS IDs in USGS v1_GU_wWs not in SDWIS:',len(combined_pws_ids) - len(usgs_check['pws_check'].unique()) )

print('\nNumber of SDWIS PWS IDs not included in the USGS v1_GU_wWs dataset:',len(sdwis_pws_ids)-len(combined_pws_ids) )

USGS v1_GU_wWs PWS IDs total: 22252

Number of rows in USGS v1_GU_wWs that were mapped to SDWIS: 25315
Number of rows in USGS v1_GU_wWs that were NOT mapped to SDWIS: 1844
Number of unique PWS IDs/WSA IDs of the ones that were mapped to the SDWIS: 18998
Number of unique PWS IDs in USGS v1_GU_wWs not in SDWIS dataset: 3254

Number of SDWIS PWS IDs not included in the USGS v1_GU_wWs dataset: 30878


In [ ]:
# Checking USDA DPC vs USGS DPC



Old Code

In [ ]:
# # Filtered for Treatment and include Facility Activity = Activity/Inactive
# sdwis_nofilter = pd.read_csv(r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024\SDWIS.csv',low_memory=False)
# sdwis_facilities_nofilter = pd.read_csv(r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024\2020_WT_Facilities_Details.csv',low_memory=False)

# Filter for Facility activity = Active
# sdwis_filtered = sdwis_filtered[sdwis_filtered['Facility Activity']=='Active']

# sdwis_no_filter_pws_ids = sdwis_nofilter['PWS ID'].unique()
# sdwis_facilities_nofilter_pws_ids = sdwis_facilities_nofilter['PWS ID'].unique()
# print('Unique PWS_ID in SDWIS no filter:', len(sdwis_no_filter_pws_ids))
# print('Unique PWS_ID in SDWIS facilities no filter:', len(sdwis_facilities_nofilter_pws_ids))

In [7]:
# Trying to check if USGS WSA data set PWS IDs can be mapped to SDWIS No filter dataset

# usgs_check = usgs_gu_wwsa_df[['GU_ID','STATE_NAME','CNTY_NM','PLACE_FIPS','PLACE_NAME',
#                               'GU_POP','WTR_TYPE_E','PWS_ID','SELLER_PWS','WSA_AGIDF']]

# # Check if the PWS ID or Seller PWS ID is in the SDWIS dataset
# check_list = []
# pws_check_list = []

# for idx,row in usgs_check.iterrows():
#     if row['SELLER_PWS'] in sdwis_no_filter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['SELLER_PWS'])
#     elif row['PWS_ID'] in sdwis_no_filter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['PWS_ID'])
#     elif row['WSA_AGIDF'] in sdwis_no_filter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['WSA_AGIDF'])
#     else:
#         check_list.append('No')
#         pws_check_list.append(row['WSA_AGIDF'])

# usgs_check['pws_check'] = pws_check_list
# usgs_check['In SDWIS'] = check_list

# print('Number of facilities mapped to SDWIS no filter:', len(usgs_check[usgs_check['In SDWIS']=='Yes']))
# print('Number of unique PWS IDs:',len(usgs_check['pws_check'].unique()))

In [80]:
# Trying to check if USGS WSA data set PWS IDs can be mapped to SDWIS Facilities no filter dataset

# usgs_check = usgs_gu_wwsa_df[['GU_ID','PLACE_FIPS','PLACE_NAME','CNTY_NM', 'STATE_NAME','PWS_ID','SELLER_PWS','WSA_AGIDF']]

# # Check if the PWS ID or Seller PWS ID is in the SDWIS dataset
# check_list = []
# pws_check_list = []

# for idx,row in usgs_check.iterrows():
#     if row['SELLER_PWS'] in sdwis_facilities_nofilter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['SELLER_PWS'])
#     elif row['PWS_ID'] in sdwis_facilities_nofilter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['PWS_ID'])
#     elif row['WSA_AGIDF'] in sdwis_facilities_nofilter_pws_ids:
#         check_list.append('Yes')
#         pws_check_list.append(row['WSA_AGIDF'])
#     else:
#         check_list.append('No')
#         pws_check_list.append(row['WSA_AGIDF'])

# usgs_check['pws_check'] = pws_check_list
# usgs_check['In SDWIS'] = check_list

# print('Number of facilities mapped to SDWIS no filter:', len(usgs_check[usgs_check['In SDWIS']=='Yes']))
# print('Number of unique PWS IDs:',len(usgs_check['pws_check'].unique()))